<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/project/step3_save_dataset_with_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install datasets transformers
# !pip install --upgrade datasets


In [4]:
# from google.colab import drive
# drive.mount("/content/gdrive")
import os
os.getcwd()

'/content'

In [6]:
import os
mode = "google-drive"
if mode=="stanford-mac" in os.getcwd():
  print("option 1")
  rgb_images_path = os.path.join("/Users/eilaarich-landkof-stanford/Downloads/train_validation_test_2023_08_14_20_18/")
elif mode=="google-drive":
  print("option 2")
  from google.colab import drive
  drive.mount("/content/gdrive")
  rgb_images_path = "/content/gdrive/MyDrive/train_validation_test_2023_08_14_20_18"
elif mode == "stanford":
  print("option 3")
  from google.colab import drive
  drive.mount("/content/gdrive")
  rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_VECT_TO_IMG_224_224/train_validation_test_2023_08_14_20_18"

option 2
Mounted at /content/gdrive


In [7]:
# import os
# rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_VECT_TO_IMG_224_224/train_validation_test"


# debug_rgb_images_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/output_folder"
output_json_file = os.path.join(rgb_images_path,'metadata.json')   # Output metadata JSON file name
output_json_file

'/content/gdrive/MyDrive/train_validation_test_2023_08_14_20_18/metadata.json'

In [8]:
import datasets

print("datasets version:", datasets.__version__)


datasets version: 2.14.4


In [9]:
import os
os.listdir(rgb_images_path)

['metadata.json',
 'tensorboard',
 'validation',
 'train',
 'test',
 'model_output_dir',
 'ds_with_pil_images']

In [10]:
os.path.basename(rgb_images_path)

'train_validation_test_2023_08_14_20_18'

In [11]:
os.path.exists(output_json_file)

True

In [12]:
# os.listdir(root_folder)

In [20]:
import os
import json

# Define the root folder
root_folder = rgb_images_path  # Replace with your root folder containing subfolders
output_json_file = os.path.join(root_folder,'metadata.json')   # Output metadata JSON file name
if (os.path.exists(output_json_file) == False):
  metadata = []
  folders_to_iterate = os.listdir(root_folder)
  folders_to_iterate = [x for x in folders_to_iterate if 'train' in x or 'validation' in x or 'test' in x]
  print ("folders_to_iterate is ",folders_to_iterate)
  # Iterate through subfolders (train, validation, test)
  for split_folder in folders_to_iterate:
    print("split_folder is ",split_folder)
    split_folder_path = os.path.join(root_folder, split_folder)

    if os.path.isdir(split_folder_path):
      for labels in os.listdir(split_folder_path):
        print("labels is {}".format(labels))
        label_folder = os.path.join(split_folder_path, labels)

        if os.path.isdir(label_folder):
          list_dir_label_folder = os.listdir(label_folder)
          # print("list_dir_label_folder is {}".format(list_dir_label_folder))
          # list_dir_label_folder = [x for x in list_dir_label_folder if ".DSto" is not in x]
          assert len(list_dir_label_folder)!=0
          for image_file in os.listdir(label_folder):
            if image_file.endswith('.jpg'):
              image_path = os.path.join(label_folder, image_file)
              # labels is required for the training
              metadata.append({"image_path": image_path, "labels": labels, "split": split_folder})

  # Save metadata as JSON file
  with open(output_json_file, 'w') as json_file:
      json.dump(metadata, json_file, indent=4)

  print("Metadata JSON file created:", output_json_file)
else:
  print("Nothing to do. Metadata JSON file was created:", output_json_file)


folders_to_iterate is  ['validation', 'train', 'test']
split_folder is  validation
labels is 
labels is 
split_folder is  train
labels is 
labels is 
split_folder is  test
labels is 
labels is 
Metadata JSON file created: /content/gdrive/MyDrive/train_validation_test_2023_08_14_20_18/metadata.json


In [21]:
output_json_file

'/content/gdrive/MyDrive/train_validation_test_2023_08_14_20_18/metadata.json'

In [22]:
# Function to load the image using PIL and add it to the dataset
# TODO: bug here: return the no array not the object type. than will be able to shorten step 4
#  the training
def load_pil_image(example):
    image_path = example["image_path"]
    pil_image = Image.open(image_path)
    example["pil_image"] = pil_image
    return example

In [ ]:
from datasets import Dataset
from PIL import Image
import json

# Load the metadata from metadata.json
with open(output_json_file, "r") as metadata_file:
    metadata = json.load(metadata_file)

# Organize the dataset entries by split
datasets_by_split = {"train": [], "test": [], "validation": []}

for entry in metadata:
    image_path = entry["image_path"]
    labels = entry["labels"]
    split = entry["split"]

    datasets_by_split[split].append({"image_path": image_path, "labels": labels})

# Convert the organized datasets into Dataset objects
dataset_objects = {}

for split, entries in datasets_by_split.items():
    dataset_objects[split] = Dataset.from_dict({"image_path": [entry["image_path"] for entry in entries],
                                                "labels": [entry["labels"] for entry in entries]})

# Apply the load_pil_image method to all images in all datasets
for split in dataset_objects.keys():
    dataset_objects[split] = dataset_objects[split].map(load_pil_image)

# Create a single dataset with subdatasets
combined_dataset = dataset_objects

# Access the train, validation, and test splits
train_dataset = combined_dataset["train"]
validation_dataset = combined_dataset["validation"]
test_dataset = combined_dataset["test"]

print("Datasets loaded and split, and images loaded as PIL images.")


Map:   0%|          | 0/30559 [00:00<?, ? examples/s]

In [ ]:
ds = combined_dataset

In [ ]:
ds["train"]

In [ ]:
ds["validation"]

In [ ]:
ds["test"]

In [ ]:
ds

In [ ]:
# Apply the function to each example in the dataset
# ds = ds.map(load_pil_image)
train_dataset

In [ ]:
type(ds)

In [ ]:
for val in ["train","validation","test"]:

  ds_with_pil_images_fld = os.path.join(rgb_images_path,'ds_with_pil_images',val)   # Output metadata JSON file name
  print("ds_with_pil_images_fld is {}".format(ds_with_pil_images_fld))
  os.makedirs(ds_with_pil_images_fld,exist_ok=True)
  ds[val].save_to_disk(ds_with_pil_images_fld)
  print("Datasets saved to disk with PIL images. ",val)
# will be used by step 4